In [160]:
import time
print(time.ctime())


Mon May 11 12:41:28 2020


<sub>Bei diesem IPython-Notebook handelt es sich um eine Beispiel-Rechnung als Ergänzung zur Dissertationsschrift von C. Knoll. Allgemeine Hinweise zu den Beispielen sind in der [readme.md-Datei des entsprechenden Repositoriums](https://github.com/cknoll/beispiele/blob/master/readme.md) zu finden.</sub>

---

**Betrachtetes System**: Pendel mit horizontal verschieblicher Aufhängung ("Wagen-Pendel-System")

￼ <img src="pendel_wagen.png">

**Betrachtete Fragen**:
* Herleitung der Modellgleichungen
    * partiell linearisiert
    * LBI-NF 
* Anwendung der Ruled-Manifold-Bedingung zum Nachweis der Nichtflachheit
* Untersuchung des Tangentialsystems zum Nachweis der Nichtflachheit




[//]: # (custom_config_start)
$$
\newcommand{\d}{\mathrm{d}}
\newcommand{\col}{\mathrm{col}}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Ddt}{\left(\tfrac{d}{dt}\right)}
\newcommand{\ddt}{\tfrac{d}{dt}}
$$
[//]: # (custom_config_end)

In [161]:
%load_ext ipydex.displaytools

import sympy as sp
from sympy import sin, cos, pi
from sympy.interactive import printing
import pickle

import symbtools as st
import symbtools.modeltools as mt

printing.init_printing(1)

import sys
import os
import pickle
import numpy as np
from scipy.integrate import odeint


from matplotlib import pyplot as plt

import symbtools.visualisation as vt

np.set_printoptions(precision=3, suppress=True)

The ipydex.displaytools extension is already loaded. To reload it, use:
  %reload_ext ipydex.displaytools


In [162]:
t = sp.Symbol('t')
np = 1
nq = 1
n = np + nq
pp = st.symb_vector("p1:{0}".format(np+1))
qq = st.symb_vector("q1:{0}".format(nq+1))
aa = st.symb_vector("a1:{0}".format(nq+1))
# ww = st.symb_vector("w1:{0}".format(nq+1))

ttheta = st.row_stack(pp, qq) ##:T
tthetad = st.time_deriv(ttheta, ttheta) ##:T
tthetadd = st.time_deriv(ttheta, ttheta, order=2) ##:T
st.make_global(ttheta, tthetad)

ttheta.T := [p₁  q₁]

---


tthetad.T := [ṗ₁  q̇₁]

---


tthetadd.T := [p̈₁  q̈₁]

---


In [163]:
params = sp.symbols('m1, m2, s2, g')
st.make_global(params)

tau1 = sp.Symbol("tau1")

### Festlegung der Geometrie des mechanischen Systemes

In [164]:
#Einheitsvektoren

ex = sp.Matrix([1,0])
ey = sp.Matrix([0,1])


# Koordinaten der Schwerpunkte und Gelenke

S1 = ex*q1 # Schwerpunkt Wagen
G2 = S1 # Pendel-Gelenk

# Schwerpunkt des Pendels (Pendel zeigt für kleine Winkel nach unten)
S2 = G2 + mt.Rz(p1)*(-ey)*s2

# Zeitableitungen der Schwerpunktskoordinaten
Sd1, Sd2  = st.col_split(st.time_deriv(st.col_stack(S1, S2), ttheta)) ##

In [165]:
# Energie
T_rot = 0 # (Punktmassenmodell)
T_trans = ( m1*Sd1.T*Sd1  +  m2*Sd2.T*Sd2 )/2

T = T_rot + T_trans[0]

V = m2*g*S2[1]

In [166]:
V

In [167]:
mod = mt.generate_symbolic_model(T, V, ttheta, [0, tau1])

In [168]:
mod.MM.simplify()
mod.MM

⎡        2                   ⎤
⎢   m₂⋅s₂       m₂⋅s₂⋅cos(p₁)⎥
⎢                            ⎥
⎣m₂⋅s₂⋅cos(p₁)     m₁ + m₂   ⎦

Bewegungsgleichungn als DGLn-System 2. Ordnung (alles auf einer Seite)

In [169]:
mod.eqns

⎡       m₂⋅s₂⋅(g⋅sin(p₁) + p̈₁⋅s₂ + q̈₁⋅cos(p₁))        ⎤
⎢                                                     ⎥
⎢           ⎛                  2                ⎞     ⎥
⎣m₁⋅q̈₁ + m₂⋅⎝p̈₁⋅s₂⋅cos(p₁) - ṗ₁ ⋅s₂⋅sin(p₁) + q̈₁⎠ - τ₁⎦

### Zustandsdarstellung mit Kraft als Eingang

In [170]:
mod.calc_state_eq()

In [177]:
mod.f ##:
mod.g ##:

(mod.f) := ⎡                    ṗ₁                     ⎤
⎢                                           ⎥
⎢                    q̇₁                     ⎥
⎢                                           ⎥
⎢ ⎛                   2           ⎞         ⎥
⎢-⎝g⋅m₁ + g⋅m₂ + m₂⋅ṗ₁ ⋅s₂⋅cos(p₁)⎠⋅sin(p₁) ⎥
⎢───────────────────────────────────────────⎥
⎢              ⎛           2    ⎞           ⎥
⎢           s₂⋅⎝m₁ + m₂⋅sin (p₁)⎠           ⎥
⎢                                           ⎥
⎢         ⎛              2   ⎞              ⎥
⎢      m₂⋅⎝g⋅cos(p₁) + ṗ₁ ⋅s₂⎠⋅sin(p₁)      ⎥
⎢      ───────────────────────────────      ⎥
⎢                         2                 ⎥
⎣              m₁ + m₂⋅sin (p₁)             ⎦

---


(mod.g) := ⎡          0          ⎤
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢      -cos(p₁)       ⎥
⎢─────────────────────⎥
⎢   ⎛           2    ⎞⎥
⎢s₂⋅⎝m₁ + m₂⋅sin (p₁)⎠⎥
⎢                     ⎥
⎢         1           ⎥
⎢  ────────────────   ⎥
⎢             2       ⎥
⎣  m₁ + m₂⋅sin (p₁)   ⎦

---


### Bestimmung der partiell lineariserten Systemgleichungen (Beschleunigung als Eingang)

In [172]:
mod.calc_coll_part_lin_state_eq(simplify=True)

In [179]:
xx = mod.x ##:T
f = mod.ff ##:
G = mod.gg 
g1, = st.col_split(G) ##:

xx.T := [p₁  q₁  ṗ₁  q̇₁]

---


f := ⎡    ṗ₁     ⎤
⎢           ⎥
⎢    q̇₁     ⎥
⎢           ⎥
⎢-g⋅sin(p₁) ⎥
⎢───────────⎥
⎢     s₂    ⎥
⎢           ⎥
⎣     0     ⎦

---


(g1,) := ⎛⎡    0    ⎤ ⎞
⎜⎢         ⎥ ⎟
⎜⎢    0    ⎥ ⎟
⎜⎢         ⎥ ⎟
⎜⎢-cos(p₁) ⎥,⎟
⎜⎢─────────⎥ ⎟
⎜⎢    s₂   ⎥ ⎟
⎜⎢         ⎥ ⎟
⎝⎣    1    ⎦ ⎠

---


In [174]:
import ipydex
#ipydex.save_current_nb_as_html()

In [175]:
simmod = st.SimulationModel(mod.f, mod.g, mod.x, model_parameters=params)



TypeError: 'Symbol' object is not subscriptable

In [176]:
%time rhs = simmod.create_simfunction()
xx0=np.array(1,1,1,1)
tt = np.linspace(0, 5, 1000)
data_f = odeint(rhs, xx0, tt) 

ValueError: The following unexpected symbols where found in `f`: {m2, g, s2, m1}